In [14]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install --upgrade web3
# !{sys.executable} -m pip install --upgrade python-dotenv
# !{sys.executable} -m pip install --upgrade watermark

## Load environment variables

In [2]:
# To read environment property file
import os
from pathlib import Path
from dotenv import load_dotenv

dotenv_path = Path('.env/uniswap')
load_dotenv(dotenv_path=dotenv_path)

True

## Constants

In [3]:
PROVIDER_URL = os.getenv('PROVIDER_URL')
# Uniswap LP address
POOL_ADDRESS = '0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640'
# Uniswap v3 Pool  ABI path
POOL_ABI = 'assets/abi/uniswapv3_pool_twap.json'
# Path to the ERC20 ABI
ERC20_ABI = 'assets/abi/erc20.json'
# Time interval to take obeservations; 5 mins gaps in between till 30 mins (1800 seconds)
TIME_INTERVALS = [x*5*60 for x in range(0,7)]
# Date time format
DATE_TIME_FMT = '%Y.%m.%d %H:%M:%S'

In [4]:
TIME_INTERVALS

[0, 300, 600, 900, 1200, 1500, 1800]

## Load Contract

In [5]:
import json
from web3 import Web3

# Load the previously downloaded pool ABI
with open(POOL_ABI) as f:
    pool_abi = json.load(f)
assert pool_abi != None, 'Pool ABI not available'

# Create a pool contract
web3 = Web3(Web3.HTTPProvider(PROVIDER_URL))
pool_contract = web3.eth.contract(address=web3.to_checksum_address(value=POOL_ADDRESS), abi=pool_abi)

# Load ERC20 ABIs to get decimals for token0 and token1
with open(ERC20_ABI) as f:
    erc20_abi = json.load(f)
assert erc20_abi != None, 'ERC20 ABI not available'

## Observations stored

In [15]:
from datetime import datetime

_, _, _, observation_count, max_observation_count, _, _ = pool_contract.functions.slot0().call()
print(f'Current Time - {datetime.now().strftime(DATE_TIME_FMT)}')
print(f'The maximum number of observations the Pool can store at the moment is {max_observation_count}')
print(f'The actual number of observations the Pool has currently stored is {observation_count}')

Current Time - 2025.10.02 17:51:14
The maximum number of observations the Pool can store at the moment is 723
The actual number of observations the Pool has currently stored is 723


## Average Ticks

In [17]:
import time

# The current time stamp; the base time; intervals will be added later
current_time_seconds = time.time()

# Call pool method to get the cumulative ticks
tick_cumulatives, _ = pool_contract.functions.observe(TIME_INTERVALS).call()

# Place holder to hold average ticks
average_ticks = []
for idx in range(len(TIME_INTERVALS[:-1])):
    # Calculate the average tick; since time interval is constant, we can simply divide it by 300 seconds
    average_ticks.append((tick_cumulatives[idx] - tick_cumulatives[idx+1])/300)

In [18]:
average_ticks

[192445.52, 192440.56, 192444.88, 192450.0, 192452.04, 192446.12]

## Tick to Price

In [20]:
# For processing large numbers
from decimal import Decimal

def tick_to_price(tick: int, token0_decimals: int, token1_decimals: int) -> (Decimal, Decimal):
    price = Decimal(1.0001) ** tick
    # Adjust for token decimals
    token0_price = round(price * (Decimal(10) ** token0_decimals) / (Decimal(10) ** token1_decimals), token1_decimals)
    token1_price = round((1/token0_price), token0_decimals)
    return (token0_price, token1_price)

## Load Token decimals and symbols via contracts

In [21]:
# Token decimals
token0_address = pool_contract.functions.token0().call()
token0_contract = web3.eth.contract(address=token0_address, abi=erc20_abi)
decimal0 = token0_contract.functions.decimals().call()

token1_address = pool_contract.functions.token1().call()
token1_contract = web3.eth.contract(address=token1_address, abi=erc20_abi)
decimal1 = token1_contract.functions.decimals().call()

# Symbols
token0_sym = token0_contract.functions.symbol.call()
token1_sym = token1_contract.functions.symbol.call()

In [23]:
from datetime import datetime

for idx in range(len(average_ticks)):
    token0_price, token1_price = tick_to_price(int(average_ticks[idx]), decimal0, decimal1)
    print(f'price of {token0_sym} in value of {token1_sym}: {token0_price} \
and {token1_sym} in value of {token0_sym}: {token1_price} at \
{datetime.fromtimestamp(current_time_seconds + TIME_INTERVALS[idx]).strftime(DATE_TIME_FMT)}')

price of USDC in value of WETH: 0.000227699614756389 and WETH in value of USDC: 4391.750953 at 2025.10.02 17:57:36
price of USDC in value of WETH: 0.000227585799095985 and WETH in value of USDC: 4393.947267 at 2025.10.02 18:02:36
price of USDC in value of WETH: 0.000227676847071682 and WETH in value of USDC: 4392.190128 at 2025.10.02 18:07:36
price of USDC in value of WETH: 0.000227813487336006 and WETH in value of USDC: 4389.555736 at 2025.10.02 18:12:36
price of USDC in value of WETH: 0.000227859052311608 and WETH in value of USDC: 4388.677956 at 2025.10.02 18:17:36
price of USDC in value of WETH: 0.000227722384717865 and WETH in value of USDC: 4391.311821 at 2025.10.02 18:22:36


## Display Modules

In [24]:
from watermark import watermark

%load_ext watermark
%watermark --iversions

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
web3     : 7.13.0
watermark: 2.5.0
json     : 2.0.9
sys      : 3.13.7 (main, Aug 15 2025, 12:34:02) [GCC 15.2.1 20250813]
decimal  : 1.70

